In [1]:
import os
import pickle as pkl

from pydeseq2.dds import DeseqDataSet
from pydeseq2.ds import DeseqStats
from pydeseq2.utils import load_example_data

import pandas as pd
import numpy as np

# Load data

In [2]:
data_file = 'GSE152800_exons.counts.tsv'
df_count = pd.read_csv(data_file, sep='\t', index_col=0)
count_df = df_count.transpose()
count_df.head()

Geneid,ENSMUSG00000051951.5,ENSMUSG00000102851.1,ENSMUSG00000103377.1,ENSMUSG00000104017.1,ENSMUSG00000103025.1,ENSMUSG00000103201.1,ENSMUSG00000103147.1,ENSMUSG00000103161.1,ENSMUSG00000102331.1,ENSMUSG00000102348.1,...,ENSMUSG00000064363.1,ENSMUSG00000064364.1,ENSMUSG00000064365.1,ENSMUSG00000064366.1,ENSMUSG00000064367.1,ENSMUSG00000064368.1,ENSMUSG00000064369.1,ENSMUSG00000064370.1,ENSMUSG00000064371.1,ENSMUSG00000064372.1
MBD725,6901,121,1443,1087,282,612,468,1679,1771,57,...,130517,35,30,27,195867,29760,876,298981,70,1282
MBD726,5073,89,977,723,281,535,302,1258,1273,50,...,68153,10,26,12,107045,16756,311,172418,41,755
MBD731,5072,77,1091,909,322,504,308,1342,1310,38,...,98206,13,19,6,145749,21671,249,237163,28,977
MBD732,5675,94,1131,969,363,565,366,1329,1455,47,...,113934,20,33,17,176437,26418,332,271334,50,1055
MBD743,4195,84,816,717,178,393,220,836,814,11,...,96585,25,56,17,160838,24645,286,213127,33,699


In [3]:
sample_info = pd.read_csv('sample_info.csv')
sample_info.head()

,sample,age,genotype
0,MBD725,15,MM2
1,MBD726,15,WT
2,MBD731,15,MM2
3,MBD732,15,WT
4,MBD743,12,WT


In [4]:
count_df['sample_name'] = count_df.index.values

count_df_ko = count_df[count_df['sample_name'].str.startswith('MM')].copy()
count_df_ko.drop(columns='sample_name', inplace=True)

count_df_mm2 = count_df[count_df['sample_name'].str.startswith('MBD')].copy()
count_df_mm2.drop(columns='sample_name', inplace=True)

In [5]:
# gene ID to gene
idmap = pd.read_excel('idmap.xlsx', index_col=0)

id_to_gene = {gene_id:gene_symbol for (gene_id, gene_symbol) in 
             zip(idmap.index.values, idmap['symbol'].values)}

# Calculate DEG for KO

In [6]:
clinical_df_ko = sample_info[sample_info['sample'].str.startswith('MM')].copy()
clinical_df_ko = clinical_df_ko.loc[:, ['sample', 'genotype']].copy()
clinical_df_ko.set_index('sample', inplace=True)
clinical_df_ko

,genotype
sample,
MM180,KO
MM181,KO
MM360,WT
MM361,WT
MM362,WT
MM363,KO
MM464,KO


In [7]:
# generate count df in terms of counts-per-million
count_df_cpm = count_df_ko.div(count_df_ko.sum(axis=1), axis=0)*1000000
count_df_cpm.head()

# filter out lowly expressed genes.
# a gene is expressed if it has >=1 cpm count in all collected samples
sel_genes = []
for col in count_df_cpm.columns:
    gene_counts = count_df_cpm[col].values
    if np.count_nonzero(gene_counts>=1)>=len(count_df_cpm):
        sel_genes.append(col)
        
### filter by gene count
count_df_filtered = count_df_ko.loc[:, sel_genes]

print(len(sel_genes))

18078


In [8]:
dds = DeseqDataSet(
            counts=count_df_filtered,
            clinical=clinical_df_ko,
            design_factors="genotype",  # compare samples based on the "condition"
            refit_cooks=True,
            n_cpus=16,
        )

dds.deseq2()
stat_res = DeseqStats(dds, n_cpus=16)

stat_res.summary()
result_df = stat_res.results_df

Fitting size factors...
... done in 0.00 seconds.

Fitting dispersions...
... done in 6.34 seconds.

Fitting dispersion trend curve...
... done in 4.24 seconds.

Fitting MAP dispersions...
... done in 6.91 seconds.

Fitting LFCs...
... done in 0.83 seconds.

Refitting 0 outliers.

Running Wald tests...
... done in 0.71 seconds.

Log2 fold change & Wald test p-value: genotype WT vs KO


,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
Geneid,,,,,,
ENSMUSG00000051951.5,4596.960938,0.034834,0.075914,0.458859,0.646336,0.822664
ENSMUSG00000102851.1,73.734116,0.008860,0.126931,0.069800,0.944353,0.978247
ENSMUSG00000103377.1,877.455261,-0.097266,0.060638,-1.604049,0.108703,0.291374
ENSMUSG00000104017.1,769.533020,-0.009786,0.079902,-0.122472,0.902526,0.958916
ENSMUSG00000103025.1,235.822067,0.068009,0.074474,0.913187,0.361144,0.607430
...,...,...,...,...,...,...
ENSMUSG00000064367.1,165509.750000,-0.118231,0.051982,-2.274469,0.022938,0.098725
ENSMUSG00000064368.1,22842.125000,-0.193499,0.068692,-2.816918,0.004849,0.030203
ENSMUSG00000064369.1,699.747070,-0.480455,0.153861,-3.122647,0.001792,0.013894


In [9]:
result_df['log2FoldChange'] = -result_df['log2FoldChange']

In [11]:
# calculate gene to rp correspondance
df_gene = pd.read_csv(r'resources/gene_info.csv', index_col=0)
df_gene['position'] = (df_gene['start'] + df_gene['end'])/2

df_rp = pd.read_csv(r"E:\DNA_analysis\Postanalysis_MeCP2\radial_position\all_rp_bulk_600pts_MOp_data.csv", 
                    index_col=0)
df_rp['chr'] = df_rp['loci_name'].apply(lambda x: x.split('_')[0])
df_rp['start'] = df_rp['loci_name'].apply(lambda x: int(x.split('_')[1]))
df_rp['end'] = df_rp['loci_name'].apply(lambda x: int(x.split('_')[2]))
df_rp['position'] = (df_rp['start'] + df_rp['end'])/2

rps = []

for i, row in df_gene.iterrows():
    df = df_rp[df_rp['chr']==row['chr']].copy()
    if len(df)>0:
        df['distance'] = np.abs(df['position']-row['position'])
        df.sort_values('distance', inplace=True)
        if df['distance'].values[0]>=3000000:
            rps.append(-1)
        else:
            rps.append(df['norm_RP'].values[0])
    else:
        rps.append(-1)
        
df_gene['radial_position'] = rps

gene_to_rp = {gene:rp for (gene, rp) in zip(df_gene['gene'].values, df_gene['radial_position'].values)}

In [15]:
result_df['gene_id'] = result_df.index.values
result_df['gene_id'] = result_df['gene_id'].apply(lambda x: x.split('.')[0])
result_df['gene_symbol'] = result_df['gene_id'].apply(lambda x: id_to_gene[x] if x in id_to_gene.keys() else 'Not_found')

result_df['radial_position'] = result_df['gene_symbol'].apply(lambda x: gene_to_rp[x] if x in gene_to_rp.keys() else -1)
result_df = result_df[result_df.radial_position!=-1].copy()
result_df

,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,gene_id,gene_symbol,radial_position
Geneid,,,,,,,,,
ENSMUSG00000051951.5,4596.960938,-0.034834,0.075914,0.458859,0.646336,0.822664,ENSMUSG00000051951,Xkr4,0.816454
ENSMUSG00000102331.1,1054.517578,-0.065015,0.081316,0.799540,0.423978,0.664042,ENSMUSG00000102331,Gm19938,0.816454
ENSMUSG00000025902.13,137.877701,0.148976,0.234180,-0.636160,0.524672,0.747444,ENSMUSG00000025902,Sox17,0.816454
ENSMUSG00000033845.13,1123.441040,0.014950,0.032770,-0.456230,0.648224,0.823454,ENSMUSG00000033845,Mrpl15,0.816454
ENSMUSG00000025903.14,780.804626,0.097723,0.071647,-1.363964,0.172579,0.394600,ENSMUSG00000025903,Lypla1,0.816454
...,...,...,...,...,...,...,...,...,...
ENSMUSG00000087201.1,246.761475,0.031288,0.140947,-0.221985,0.824325,0.923218,ENSMUSG00000087201,Gm15261,0.813251
ENSMUSG00000031352.10,880.726562,0.080400,0.084533,-0.951114,0.341546,0.589866,ENSMUSG00000031352,Hccs,0.813251
ENSMUSG00000087159.7,186.408707,0.069706,0.139137,-0.500985,0.616382,0.806545,ENSMUSG00000087159,Gm15246,0.813251


In [16]:
result_df.to_csv(r'resources\Tillotson_KO_deg_rp.csv')

# Calculate DEG for MM2

In [17]:
clinical_df_mm2 = sample_info[sample_info['sample'].str.startswith('MBD')].copy()
clinical_df_mm2 = clinical_df_mm2.loc[:, ['sample', 'genotype']].copy()
clinical_df_mm2.set_index('sample', inplace=True)
clinical_df_mm2

,genotype
sample,
MBD725,MM2
MBD726,WT
MBD731,MM2
MBD732,WT
MBD743,WT
MBD744,MM2
MBD745,WT
MBD746,MM2
MBD755,WT


In [18]:
# generate count df in terms of counts-per-million
count_df_cpm = count_df_mm2.div(count_df_mm2.sum(axis=1), axis=0)*1000000
count_df_cpm.head()

# filter out lowly expressed genes.
# a gene is expressed if it has >=1 cpm count in all collected samples
sel_genes = []
for col in count_df_cpm.columns:
    gene_counts = count_df_cpm[col].values
    if np.count_nonzero(gene_counts>=1)>=len(count_df_cpm):
        sel_genes.append(col)
        
### filter by gene count
count_df_filtered = count_df_mm2.loc[:, sel_genes]

print(len(sel_genes))

17588


In [19]:
dds = DeseqDataSet(
            counts=count_df_filtered,
            clinical=clinical_df_mm2,
            design_factors="genotype",  # compare samples based on the "condition"
            refit_cooks=True,
            n_cpus=16,
        )

dds.deseq2()
stat_res = DeseqStats(dds, n_cpus=16)

stat_res.summary()
result_df = stat_res.results_df

Fitting size factors...
... done in 0.00 seconds.

Fitting dispersions...
... done in 6.61 seconds.

Fitting dispersion trend curve...
... done in 4.14 seconds.

Fitting MAP dispersions...
... done in 7.86 seconds.

Fitting LFCs...
... done in 0.71 seconds.

Refitting 3 outliers.

Fitting dispersions...
... done in 0.01 seconds.

Fitting MAP dispersions...
... done in 0.01 seconds.

Fitting LFCs...
... done in 0.00 seconds.

Running Wald tests...
... done in 0.73 seconds.

Log2 fold change & Wald test p-value: genotype WT vs MM2


,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
Geneid,,,,,,
ENSMUSG00000051951.5,4090.489014,-0.053954,0.048097,-1.121778,0.261957,0.572690
ENSMUSG00000102851.1,72.589165,0.052473,0.128803,0.407391,0.683720,0.884081
ENSMUSG00000103377.1,843.191650,-0.056425,0.043008,-1.311948,0.189538,0.477045
ENSMUSG00000104017.1,691.183655,-0.082897,0.051951,-1.595698,0.110556,0.349221
ENSMUSG00000103025.1,220.302826,-0.099738,0.115835,-0.861037,0.389218,0.699462
...,...,...,...,...,...,...
ENSMUSG00000064367.1,135909.656250,0.075762,0.102350,0.740223,0.459165,0.755244
ENSMUSG00000064368.1,19862.875000,0.096308,0.105375,0.913959,0.360738,0.674319
ENSMUSG00000064369.1,434.733704,-0.111261,0.309637,-0.359328,0.719349,0.902388


In [20]:
result_df['log2FoldChange'] = -result_df['log2FoldChange']

result_df['gene_id'] = result_df.index.values
result_df['gene_id'] = result_df['gene_id'].apply(lambda x: x.split('.')[0])
result_df['gene_symbol'] = result_df['gene_id'].apply(lambda x: id_to_gene[x] if x in id_to_gene.keys() else 'Not_available')

result_df['radial_position'] = result_df['gene_symbol'].apply(lambda x: gene_to_rp[x] if x in gene_to_rp.keys() else -1)
result_df = result_df[result_df.radial_position!=-1].copy()
result_df

,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,gene_id,gene_symbol,radial_position
Geneid,,,,,,,,,
ENSMUSG00000051951.5,4090.489014,0.053954,0.048097,-1.121778,0.261957,0.572690,ENSMUSG00000051951,Xkr4,0.816454
ENSMUSG00000102331.1,954.203918,0.115978,0.082100,-1.412634,0.157763,0.428532,ENSMUSG00000102331,Gm19938,0.816454
ENSMUSG00000025902.13,126.698853,-0.193941,0.152630,1.270662,0.203849,0.499206,ENSMUSG00000025902,Sox17,0.816454
ENSMUSG00000033845.13,955.557861,-0.057941,0.038656,1.498905,0.133898,0.390095,ENSMUSG00000033845,Mrpl15,0.816454
ENSMUSG00000025903.14,636.065674,-0.067835,0.083763,0.809843,0.418030,0.723582,ENSMUSG00000025903,Lypla1,0.816454
...,...,...,...,...,...,...,...,...,...
ENSMUSG00000087201.1,239.379395,0.320489,0.132255,-2.423264,0.015382,0.090844,ENSMUSG00000087201,Gm15261,0.813251
ENSMUSG00000031352.10,794.103333,-0.023348,0.075800,0.308014,0.758071,0.916667,ENSMUSG00000031352,Hccs,0.813251
ENSMUSG00000087159.7,148.268051,0.144121,0.073383,-1.963960,0.049535,0.208675,ENSMUSG00000087159,Gm15246,0.813251


In [21]:
result_df.to_csv(r'resources\Tillotson_MM2_deg_with_rp.csv')